# Modelo de clasificación por formato

#### EN ESTE NOTEBOOK VAMOS A COMPROBAR SI PUDIERAMOS ESTABLECER ALGUNA CONEXION ENTRE LOS DATOS QUE PODEMOS OBTENER DEL FORMATO Y NUESTRO TARGET DE ANALISIS DE SENTIMIENTO, DE FORMA BINARIA.

![](images/text.jpg)

In [ ]:
# Importo las librerias que voy a utilizar en este notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('../elmundo/input/df_elmundo_fin.csv',index_col=[0], parse_dates=True)
df.sample(8)

## Posibles features

     - dia de la semana del 1 al 7
     - mes
     - año
     - numero de adjetivos, verbos, sustantivos)
     - numero de palabras relativas al maximo
     - numero de stopwords
     - numero de palabras unicas
     - autores = LabelEncoder

In [ ]:
df['Fecha'] = pd.to_datetime(df.Fecha)
numeric_df = df[['Fecha', 'Texto','Autores','Coincidencia_elmundo_relativa','Scores_zero',
                 'Analisis_sentimiento_scd']]
numeric_df

In [ ]:
# dia de la semana, dia y mes en numero.
numeric_df['dia'] = numeric_df.Fecha.dt.day
numeric_df['dia_semana'] = numeric_df.Fecha.dt.dayofweek
numeric_df['mes'] = numeric_df.Fecha.dt.month

In [ ]:
# número de stopwords.
stop_spa = stopwords.words('spanish')

def counter_stopwords(fila):
    counter = 0
    for i in fila.split():
        if i in stop_spa:
            counter += 1
    return counter

numeric_df['num_stop_words'] = numeric_df['Texto'].apply(lambda x: counter_stopwords(x))    

In [ ]:
# numero de palabras
def counter_words(fila):
    counter = 0
    for i in fila.split():
           counter += 1
    return counter
numeric_df['num_palabras'] = numeric_df.Texto.apply(lambda x: counter_words(x))

In [ ]:
numeric_df['len_total'] = numeric_df.Texto.apply(lambda x: len(x))
numeric_df

In [ ]:
# Numero de palabras únicas.
def counter_uniques(fila):
    for i in fila:
        
        return len(set(i))
    
numeric_df['unique'] = numeric_df['Texto'].apply(lambda x: counter_uniques(x.split()))

In [ ]:
# Contamos los signos de excl, interrogacion y puntuaciones
numeric_df['num_exclamacion'] = numeric_df['Texto'].apply(lambda x: x.count('!'))
numeric_df['num_interrogaciones'] = numeric_df['Texto'].apply(lambda x: x.count('?'))
numeric_df['num_puntuaciones'] = numeric_df['Texto'].apply(lambda x: sum(x.count(i) for i in '.,;:'))

In [ ]:
# Seleccionamos X e Y
cols = ['num_stop_words', 'num_palabras', 'len_total',
       'unique','num_exclamacion', 'num_interrogaciones',
       'num_puntuaciones']

In [ ]:
# Creamos el target y normalizamos los datos.
X = numeric_df[cols]
y = np.where(numeric_df['Analisis_sentimiento_scd'] > 0,1,0)

X['num_stop_words'] = X.num_stop_words/X.num_stop_words.max()
X['len_total'] = X.len_total/X.len_total.max()
X['unique'] = X['unique']/X['unique'].max()
X['num_puntuaciones'] = X.num_puntuaciones/X.num_puntuaciones.max()
X['num_palabras'] = X.num_palabras/X.num_palabras.max()

In [ ]:
# Separamos en train y test

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size=0.15,
                                                   random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Creamos el modelo con keras
model = keras.models.Sequential([
    keras.layers.Dense(input_dim = 7,
                      units=3,
                      activation='sigmoid'),
    keras.layers.Dense(units=1, activation='sigmoid')
])
opt = keras.optimizers.Adam(learning_rate=0.002)
model.compile(loss = 'binary_crossentropy', 
             optimizer=opt, metrics='accuracy')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 4,
                                                 restore_best_weights=True)
history = model.fit(X_train,
                   y_train,
                   epochs=50,
                   batch_size=128)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid()
plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Podemos comprobar que le cuesta mucho encontrar patrones en los datos, y el accuracy siempre se mantiene
# en el porcentaje de balanceo del target.


Vamos a probar con otros modelos por si pudieran encontrar algo diferente

## Logistic Regression

In [ ]:
lg = LogisticRegression()
params = {
    'C': [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'penalty': ['l2','l1'],
    'solver':['liblinear'],
    'max_iter':[50,100,200]
}
grid = GridSearchCV(estimator=lg,param_grid=params, cv=4)
grid.fit(X_train, y_train)

In [ ]:
y_pred_lg = grid.best_estimator_.predict(X_test)
print(classification_report(y_pred_lg,y_test))

## AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier()
params = {
    'n_estimators' : [10,20,30,40,50,60,70,80,200],
    'learning_rate' : [0.1,0.3,0.4,0.7,1,1.5],
    'algorithm' : ['SAMME','SAMME.R']
}
grid2 = GridSearchCV(estimator=ada,
                    param_grid = params)
grid2.fit(X_train,y_train)

In [ ]:
y_pred_ada = grid2.best_estimator_.predict(X_test)
print(classification_report(y_pred_ada,y_test))

**Como podemos ver, todos los modelos nos dan los mismos resultados con estos datos;
Luego lo que da que pensar que el problema viene del etiquetado o de la extraccion de los mismos**